In [8]:
import torch
import cv2
import numpy as np
import os

In [4]:
BUCKET_NAME = "eth-capstone-camera-traps"
SM_ROLE = "SagemakerExecutionRole"

In [3]:
# def prepare_train_data(frames, sequence_length: int = 5):
#     sequence_length = 5  # 3-5 frames as input, adjust as needed
#     X, y = [], []

#     for i in range(len(frames) - sequence_length):
#         X.append(frames[i:i + sequence_length])  # Input sequence
#         y.append(frames[i + sequence_length])    # Next frame to predict

#     # Convert to numpy arrays and normalize
#     X = np.array(X) / 255.0
#     y = np.array(y) / 255.0

#     X = X.reshape((X.shape[0],X.shape[1],X.shape[-1],X.shape[2],X.shape[3]))
#     y = y.reshape((y.shape[0],y.shape[-1],y.shape[1],y.shape[2]))

#     return X,y

In [5]:
frames = extract_frames(video_path="/Users/chrishon/capstone-camera-traps/resources/videos/FH102_02 (1).avi")

In [6]:
X,y = prepare_train_data(frames)

In [ ]:
import sagemaker
from sagemaker.pytorch import PyTorch

# Initialize SageMaker session
sagemaker_session = sagemaker.Session()
role = SM_ROLE  # Replace with your IAM role

# Define PyTorch Estimator
estimator = PyTorch(
    entry_point="train_convlstm.py",
    role=role,
    framework_version="2.5",
    py_version="py311",
    instance_count=1,
    instance_type="ml.p3.2xlarge",  # Select a GPU instance type
    hyperparameters={
        "batch-size": 8,
        "lr": 0.001,
        "epochs": 20,
    },
    source_dir=".",  # Directory containing `train.py`
    use_spot_instances=True,
    output_path=f"s3://{BUCKET_NAME}",
)

# Start training job with input from S3
estimator.fit({"training": f"s3://{BUCKET_NAME}"})
